In [3]:
from dotenv import load_dotenv
import logging
from rich.console import Console
from rich.prompt import Prompt

from stops_qa import (
    setup_database,
    setup_cache,
    load_data_to_db,
    create_sql_generator,
    generate_sql_query,
    execute_query,
    display_query_result,
    display_welcome,
    TEXTOS
)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
console = Console()
load_dotenv()
setup_cache()

In [4]:
try:
    conn = setup_database(db_name="stops.db")
    load_data_to_db(conn, csv_file="simulated_stops.csv")
    llm, prompt = create_sql_generator()
    display_welcome()

    while True:
        question = Prompt.ask(TEXTOS["prompt_pregunta"])
        
        if question.lower() == 'exit':
            console.print(TEXTOS["despedida"])
            break
            
        try:
            sql_query = generate_sql_query(question)
            
            with console.status(TEXTOS["ejecutando_query"]) as status:
                results, error = execute_query(conn, sql_query)
                display_query_result(sql_query, results, error)
            
        except Exception as e:
            logger.error(f"Error: {str(e)}")
            console.print(f"{TEXTOS['error']} {str(e)}")

except Exception as e:
    console.print(f"{TEXTOS['error_fatal']} {str(e)}")
finally:
    if 'conn' in locals():
        conn.close()

✓ Cargados 1422 registros en la base de datos.

╭─────────────────────────────────── Sistema de Análisis de Paradas de Máquina ───────────────────────────────────╮
│                                                                                                                 │
│ ¡Bienvenido a StopQuery!                                                                                        │
│                                                                                                                 │
│ Este sistema te permite hacer preguntas en lenguaje natural sobre las paradas                                   │
│ de máquina y las convierte en consultas SQL para su análisis.                                                   │
│                                                                                                                 │
│ Ejemplos de Preguntas:                                                                                          │
│ • ¿Cuántas paradas ocurrieron el 1 de enero?                                                                    │
│ • Lista todas las paradas causadas por 'Die Head Cleaning'                                                      │
│ • ¿Cuál es la duración promedio de cada tipo de parada?                                                         │
│ • ¿Qué hora del día tuvo la duración total de paradas más larga?                                                │
│                                                                                                                 │
│ Escribe 'exit' para salir                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Tu pregunta:

Generando nueva consulta SQL...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:stops_qa:Ejecutando SQL: SELECT COUNT(*) 
FROM stops 
WHERE DATE(start_at) = '2023-01-01'


...

SELECT COUNT(*)                                                                                                    
FROM stops                                                                                                         
WHERE DATE(start_at) = '2023-01-01'                                                                                

Resultados:

┏━━━━━━━━━━┓
┃ COUNT(*) ┃
┡━━━━━━━━━━┩
│ 0        │
└──────────┘

Tu pregunta:

¡Hasta luego!